#### Import the following packages:

In [39]:
#Import the right libraries 
# load the libraries needed to run the analysis
import pandas as pd
import math
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import re
import scipy
import pickle

#### ###Load the housing dataset

In [40]:
# Training data
house = pd.read_csv('/Users/mofuoku/Desktop/train.csv')

# Test data 
house_test = pd.read_csv('/Users/mofuoku/Desktop/test.csv')

trainSize = house.shape[0]
house.SalePrice = house.SalePrice.apply(np.log)
print(house.shape)
print(house_test.shape)

(1460, 81)
(1459, 80)


In [41]:
#Transpose the results to make printing on the screen easier.
summary = house.describe()
summary = summary.transpose()
print(summary)

                count          mean          std          min          25%  \
Id             1460.0    730.500000   421.610009     1.000000   365.750000   
MSSubClass     1460.0     56.897260    42.300571    20.000000    20.000000   
LotFrontage    1201.0     70.049958    24.284752    21.000000    59.000000   
LotArea        1460.0  10516.828082  9981.264932  1300.000000  7553.500000   
OverallQual    1460.0      6.099315     1.382997     1.000000     5.000000   
OverallCond    1460.0      5.575342     1.112799     1.000000     5.000000   
YearBuilt      1460.0   1971.267808    30.202904  1872.000000  1954.000000   
YearRemodAdd   1460.0   1984.865753    20.645407  1950.000000  1967.000000   
MasVnrArea     1452.0    103.685262   181.066207     0.000000     0.000000   
BsmtFinSF1     1460.0    443.639726   456.098091     0.000000     0.000000   
BsmtFinSF2     1460.0     46.549315   161.319273     0.000000     0.000000   
BsmtUnfSF      1460.0    567.240411   441.866955     0.000000   

#### Some details about the data:

In [43]:
#Understanding Data 
#df.describe() 

In [44]:
#save the Id column:
train_id = house['Id']
test_id  = house_test['Id']

# Now drop the 'Id' column 
# since we can not use it as a feature to train our model.
house.drop("Id", axis = 1, inplace = True)
house_test.drop("Id", axis = 1, inplace = True)

In [45]:
# distinguish the NA and missing in train data
miss = list(house.columns[house.isnull().sum(axis = 0) >= 1])
miss.remove('LotFrontage')
miss.remove('MasVnrArea')
miss.remove('GarageYrBlt')
miss.remove('MasVnrType')

house.loc[house['MasVnrType'].isnull() == True, 'MasVnrType'] = 'None'
house.loc[house['Electrical'].isnull() == True, 'Electrical'] = 'SBrkr'

for i in range(len(miss)):
    house.loc[house[miss[i]].isnull() == True, miss[i]] = 'NA'

In [46]:
with open('/Users/mofuoku/Desktop/data_description.txt', 'r') as file:
    lines_level = file.readlines()

In [47]:
feature = ''
levels = {}
level = ''

for line in lines_level:
    if (not not re.findall(': ', line)) and (not re.findall('story:', line)):
#         features.append(re.split(':', line)[0])
        feature = re.split(':', line)[0]
        levels[feature] = []
    else:
        if line != '\n':
#             levels.append(re.split('\t', line)[0].strip())
            level = re.split('\t', line)[0].strip()
            if level != '': levels[feature].append(level)
levels['Id'] = []
levels['SalePrice'] = []
# print(levels)

In [48]:
# find ordinal and non ordinal
with open('/Users/mofuoku/Desktop/DataDocumentation.txt', 'r',encoding='cp1252') as file:
    lines = file.readlines()

In [49]:
# Convert string data to numerical data 

nom_lines = []
ord_lines = []
for line in lines:
    if re.findall('(Nominal)', line):
        nom_lines.append(re.sub(' ', '', re.split('\(Nominal\)', line)[0].strip()))
    elif re.findall('(Ordinal)', line):
        ord_lines.append(re.sub(' ', '', re.split('\(Ordinal\)', line)[0].strip()))
print(nom_lines)
print(ord_lines)

['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LandContour', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1', 'Exterior2', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition']
['LotShape', 'Utilities', 'LandSlope', 'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence']


In [50]:
def my_cat(col, levels, ord_lines):
    if not not levels[col.name]: 
        if col.name in ord_lines:
            temp = pd.Categorical(list(col.astype('str')), ordered=True, categories=levels[col.name][::-1])
            temp.name = col.name
            return temp
        else:
            temp = pd.Categorical(list(col.astype('str')), categories=levels[col.name][::-1])
            temp.name = col.name
            return temp
   
    else:
        return col

In [51]:
for col in house:
    house[col] = my_cat(house[col], levels, ord_lines)

In [52]:
miss = list(house_test.columns[house_test.isnull().sum(axis = 0) >= 1])
miss.remove('LotFrontage')
miss.remove('MasVnrArea')
miss.remove('GarageYrBlt')
miss.remove('TotalBsmtSF')
miss.remove('BsmtHalfBath')
miss.remove('BsmtFullBath')
miss.remove('GarageCars')
miss.remove('GarageArea')
miss.remove('BsmtFinSF1')
miss.remove('BsmtFinSF2')
miss.remove('BsmtUnfSF')

house_test.loc[house_test['MasVnrType'].isnull() == True, 'MasVnrType'] = 'CBlock'
house_test.loc[house_test['SaleType'].isnull() == True, 'SaleType'] = 'Oth'
house_test.loc[house_test['Functional'].isnull() == True, 'Functional'] = 'Sal'
house_test.loc[house_test['MSZoning'].isnull() == True, 'MSZoning'] = 'A'
house_test.loc[house_test['Exterior1st'].isnull() == True, 'Exterior1st'] = 'WdShing'
house_test.loc[house_test['Exterior2nd'].isnull() == True, 'Exterior2nd'] = 'Wd Shng'

for i in range(len(miss)):
    house_test.loc[house_test[miss[i]].isnull() == True, miss[i]] = 'NA'

In [53]:
for col in house_test:
    house_test[col] = my_cat(house_test[col], levels, ord_lines)

In [54]:
print(levels['OverallQual'][::-1])
print(levels['ExterQual'][::-1])
print(levels['BsmtQual'][::-1])
print(levels['KitchenQual'][::-1])
print(levels['GarageFinish'][::-1])
print(levels['FireplaceQu'][::-1])
print(levels['HeatingQC'][::-1])

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
['Po', 'Fa', 'TA', 'Gd', 'Ex']
['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
['Po', 'Fa', 'TA', 'Gd', 'Ex']
['NA', 'Unf', 'RFn', 'Fin']
['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
['Po', 'Fa', 'TA', 'Gd', 'Ex']


In [55]:
def OrdinalsToNumerics(all_data):    
    all_data['ExterQual'] = all_data['ExterQual'].map(lambda x: {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}.get(x, 0))
    all_data['BsmtQual'] = all_data['BsmtQual'].map(lambda x: {'NA':1,'Po':2, 'Fa':3, 'TA':4, 'Gd':5, 'Ex':6}.get(x, 0))
    all_data['KitchenQual'] = all_data['KitchenQual'].map(lambda x: {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}.get(x, 0))
    all_data['GarageFinish'] = all_data['GarageFinish'].map(lambda x: {'NA':1, 'Unf':2, 'RFn':3, 'Fin':4}.get(x, 0))
    all_data['FireplaceQu'] = all_data['FireplaceQu'].map(lambda x: {'NA':1,'Po':2, 'Fa':3, 'TA':4, 'Gd':5, 'Ex':6}.get(x, 0))
    all_data['HeatingQC'] = all_data['HeatingQC'].map(lambda x: {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}.get(x, 0))

    all_data['OverallQual'] = pd.to_numeric(all_data['OverallQual'])
    all_data['ExterQual'] = pd.to_numeric(all_data['ExterQual'])
    all_data['BsmtQual'] = pd.to_numeric(all_data['BsmtQual'])
    all_data['KitchenQual'] = pd.to_numeric(all_data['KitchenQual'])
    all_data['GarageFinish'] = pd.to_numeric(all_data['GarageFinish'])
    all_data['FireplaceQu'] = pd.to_numeric(all_data['FireplaceQu'])
    all_data['HeatingQC'] = pd.to_numeric(all_data['HeatingQC'])

In [56]:
# Idea taken from this script: https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
# # Add some family features directly to new columns in the dataset

def FeatureEngineering(all_data):
    # Adding total sqfootage feature 
    all_data['GarageYrBlt'][all_data['GarageYrBlt'].isnull()] = all_data['YearBuilt'][all_data['GarageYrBlt'].isnull()]
    all_data['GarageYrBlt'] = all_data['GarageYrBlt'] - all_data['YearBuilt']

    #all_data['YrSold'] = all_data['YrSold'] - all_data['YearBuilt']
    
    all_data['MoSold'] = pd.Categorical(list(all_data['MoSold'].astype('str')))
    
    all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

    # GarageYrBlt, GarageArea and GarageCars : Replacing missing data with 0 (Since No garage = no cars in such garage.)
    for col in ('GarageArea', 'GarageCars'):
        all_data[col] = all_data[col].fillna(0)

    # BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath and BsmtHalfBath : missing values are likely zero for having no basement
    for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
        all_data[col] = all_data[col].fillna(0)

    # LotFrontage : Since the area of each street connected to the house property most likely have a similar 
    # area to other houses in its neighborhood , we can fill in missing values by the median LotFrontage of the neighborhood.
    # Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
    all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
    
    all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']
    
    # Utilities : For this categorical feature all records are "AllPub", except for one "NoSeWa" and 2 NA . Since the house with 'NoSewa' is in the training set, this feature won't help in predictive modelling. We can then safely remove it.
    all_data = all_data.drop(['Utilities'], axis=1)

drop_cols1 = ["Alley", "ExterCond", "BsmtFinSF2", "Heating", "Electrical",\
              "BsmtHalfBath", "PavedDrive", "Fence", "MiscFeature", "MoSold"]

drop_cols2 = ["BsmtFinType2", "LandContour", "EnclosedPorch",\
              "SaleType", "Functional", "LotConfig",\
              "ScreenPorch", "YrSold", "LandSlope"]


In [57]:
# Specify columns to drop
def DropColumns(all_data):
    for col in [*drop_cols1, *drop_cols2]:
        all_data.drop([col], axis=1, inplace=True)

In [58]:
price = house.SalePrice
house.drop('SalePrice', axis=1, inplace=True)

In [59]:
OrdinalsToNumerics(house)
OrdinalsToNumerics(house_test)

FeatureEngineering(house)
FeatureEngineering(house_test)

DropColumns(house)
DropColumns(house_test)

In [60]:
all_data = pd.concat([house, house_test], ignore_index = True)
all_data = pd.get_dummies(all_data, drop_first=True)

In [61]:
train_clean = all_data.iloc[:trainSize,:]
train_clean = train_clean.assign(SalePrice = price.values)
train_clean.shape

(1460, 219)

In [62]:
trainSize

1460

In [63]:
test_clean = all_data.iloc[trainSize:, :]
test_clean.shape

(1459, 218)

## Machine Learning Models

In [64]:
#Load the libraries

# The usual
import numpy as np

# sklearn tools for model training and assesment
import sklearn.model_selection as ms
from sklearn.model_selection import GridSearchCV, train_test_split,KFold
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor #as gbr as rfr
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

from mlxtend.regressor import StackingRegressor
from xgboost.sklearn import XGBRegressor
# LightGBM
import lightgbm as lgb
# Lightxgboost
import xgboost as xgb

def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

#### Split the dataset into training and testing

In [65]:
rs_const = 80 #always use the same one value for all model tuning process
test_ratio_const = 0.2 

train_data   = train_clean.drop('SalePrice', axis=1)
train_target = train_clean[['SalePrice']]
test_data    = test_clean

X_train, X_test, y_train, y_test = train_test_split(train_data, train_target,\
                                                    test_size=test_ratio_const, random_state=rs_const)

In [66]:
print('X train shape = ',X_train.shape)
print('X test shape = ', X_test.shape)
print('Y train shape = ', y_train.shape)
print('Y test shape = ',y_test.shape)

X train shape =  (1168, 218)
X test shape =  (292, 218)
Y train shape =  (1168, 1)
Y test shape =  (292, 1)


In [67]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_std = scaler.transform(X_train)
X_test_std  = scaler.transform(X_test)

In [242]:
#X_train,X_valid,y_train,y_valid = train_test_split(X_train,y_train,test_size=0.10, shuffle = True,random_state=80)

#### Create Hyperparameter Search Space

In [ ]:
# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter space
C = np.logspace(0, 4, 10)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

#### Create Grid Search

In [ ]:
# Create grid search using 5-fold cross validation
clf = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)

#### View Hyperparameter Values Of Best Model

In [ ]:
# View best hyperparameters
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

### 1. Linear Regression

#### Build the model

In [80]:
model_lr = linear_model.LinearRegression()

#### Fit The Model

In [81]:
model_lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [83]:
print("RMSE train: {}".format(rmse(y_train, model_lr.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_lr.predict(X_test))))

RMSE train: 0.10262489450225752
RMSE test : 0.1273096943564471


### 2. Kernel Ridge Regression 

#### Create Hyperparameter Search Space

In [89]:
param_grid_krr = [{'alpha': np.logspace(-4, 4, 20)}]

#### Create Grid Search

In [90]:
gridsearch_krr = GridSearchCV(estimator=linear_model.Ridge(random_state=rs_const),param_grid= param_grid_krr, n_jobs=-1,cv=5, verbose=True)


#### Fit The Model

In [91]:
gridsearch_krr.fit(X_train_std,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   18.2s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=80, solver='auto', tol=0.001),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'alpha': array([1.00000e-04, 2.63665e-04, 6.95193e-04, 1.83298e-03, 4.83293e-03,
       1.27427e-02, 3.35982e-02, 8.85867e-02, 2.33572e-01, 6.15848e-01,
       1.62378e+00, 4.28133e+00, 1.12884e+01, 2.97635e+01, 7.84760e+01,
       2.06914e+02, 5.45559e+02, 1.43845e+03, 3.79269e+03, 1.00000e+04])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

In [ ]:
#### Evaluating the model and printing the results

In [92]:
print("_______________________________________")
print("krr: Regression")
print("_______________________________________")
print('Best params: {}'.format(gridsearch_krr.best_params_))
print('Best score : {}'.format(gridsearch_krr.best_score_))
print("---------")
model_krr = gridsearch_krr.best_estimator_
print("RMSE train: {}".format(rmse(y_train, model_krr.predict(X_train_std))))
print("RMSE test : {}".format(rmse(y_test,  model_krr.predict(X_test_std))))

_______________________________________
krr: Regression
_______________________________________
Best params: {'alpha': 545.5594781168514}
Best score : 0.862816990220294
---------
RMSE train: 0.11500081056347171
RMSE test : 0.09967169272824544


### 3. Lasso Regression

#### Create Hyperparameter Search Space

In [97]:
param_grid_lasso = [{'alpha': np.logspace(-3, 4, 20)}] 

#### Create Grid Search

In [98]:
gridsearch_lasso = GridSearchCV(estimator=linear_model.Lasso(random_state=rs_const, normalize=False), param_grid=param_grid_lasso, n_jobs=-1,cv=5, verbose=True)

#### Fit The Model

In [99]:
gridsearch_lasso.fit(X_train_std, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   19.5s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=80,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'alpha': array([1.00000e-03, 2.33572e-03, 5.45559e-03, 1.27427e-02, 2.97635e-02,
       6.95193e-02, 1.62378e-01, 3.79269e-01, 8.85867e-01, 2.06914e+00,
       4.83293e+00, 1.12884e+01, 2.63665e+01, 6.15848e+01, 1.43845e+02,
       3.35982e+02, 7.84760e+02, 1.83298e+03, 4.28133e+03, 1.00000e+04])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

#### Evaluate The Model And Print The Results

In [100]:
print("_______________________________________")
print("Lasso: Regression")
print("_______________________________________")
print('Best params: {}'.format(gridsearch_lasso.best_params_))
print('Best score : {}'.format(gridsearch_lasso.best_score_))
print("---------")
model_lasso = gridsearch_krr.best_estimator_
print("RMSE train: {}".format(rmse(y_train, model_lasso.predict(X_train_std))))
print("RMSE test : {}".format(rmse(y_test,  model_lasso.predict(X_test_std))))

_______________________________________
Lasso: Regression
_______________________________________
Best params: {'alpha': 0.00545559478116852}
Best score : 0.8564711534949876
---------
RMSE train: 0.11500081056347171
RMSE test : 0.09967169272824544


### 4. Elastic Net Regression

#### Create Hyperparameter Search Space

In [125]:
param_grid_enet = [{'alpha': np.logspace(-2, 4, 20), 'l1_ratio': np.linspace(0.015, 1, 20)}]

#### Create Grid Search

In [126]:
gridsearch_enet = GridSearchCV(estimator=linear_model.ElasticNet(random_state=rs_const), param_grid=param_grid_enet, n_jobs=-1,cv=5, verbose=True)

#### Fit The Model

In [127]:
gridsearch_enet.fit(X_train_std, y_train)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 478 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 1478 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:   19.2s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=80, selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'alpha': array([1.00000e-02, 2.06914e-02, 4.28133e-02, 8.85867e-02, 1.83298e-01,
       3.79269e-01, 7.84760e-01, 1.62378e+00, 3.35982e+00, 6.95193e+00,
       1.43845e+01, 2.97635e+01, 6.15848e+01, 1.27427e+02, 2.63665e+02,
       5.45559e+02, 1.12884e+03, 2.33572e+03, 4.83293e+03, 1.0... 0.53342, 0.58526, 0.63711, 0.68895,
       0.74079, 0.79263, 0.84447, 0.89632, 0.94816, 1.     ])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

#### Evaluate The Model And Print The Results

In [128]:
print('Best params: {}'.format(gridsearch_enet.best_params_))
print('Best score : {}'.format(gridsearch_enet.best_score_))

model_en = gridsearch_enet.best_estimator_
print("RMSE train: {}".format(rmse(y_train, model_en.predict(X_train_std))))
print("RMSE test : {}".format(rmse(y_test,  model_en.predict(X_test_std))))

Best params: {'alpha': 0.18329807108324356, 'l1_ratio': 0.015}
Best score : 0.8652189389746141
RMSE train: 0.1127655445071562
RMSE test : 0.0955844475998484


### 5. Gradient Boosting Regression

#### Create Hyperparameter Search Space

In [133]:
param_grid_gboost = {
                    'learning_rate' : [0.005, 0.01,0.05],
                    'n_estimators': [2000,3000,4000],
                    'max_depth' : [4,5,7],
                    'min_samples_split': [5,7,9], 
                    'min_samples_leaf' : [1,3,5],
                     'max_features' : ['sqrt'],
                    'subsample':[0.7,0.8,0.9]
}

#### Create Grid Search

In [135]:
gridsearch_gboost= GridSearchCV(estimator=GradientBoostingRegressor(random_state=rs_const), param_grid=param_grid_gboost,n_jobs=-1, cv=5,verbose=True)

#### Fit The Model

In [136]:
gridsearch_gboost.fit(X_train, y_train)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 13.8min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 25.7min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 46.8min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 64.2min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed: 92.4min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed: 117.1min
[Parallel(n_jobs=-1)]: Done 3645 out of 3645 | elapsed: 134.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_sampl...te=80, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'learning_rate': [0.005, 0.01, 0.05], 'n_estimators': [2000, 3000, 4000], 'max_depth': [4, 5, 7], 'min_samples_split': [5, 7, 9], 'min_samples_leaf': [1, 3, 5], 'max_features': ['sqrt'], 'subsample': [0.7, 0.8, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=True)

#### Evaluate The Model And Print The Results

In [141]:
print("_______________________________________")
print("gboost: Regression")
print("_______________________________________")
print('Best params: {}'.format(gridsearch_gboost.best_params_))
print('Best score : {}'.format(gridsearch_gboost.best_score_))
print("---------")
model_gboost = gridsearch_gboost.best_estimator_
print("RMSE train: {}".format(rmse(y_train, model_gboost.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_gboost.predict(X_test))))

_______________________________________
gboost: Regression
_______________________________________
Best params: {'learning_rate': 0.005, 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 4000, 'subsample': 0.7}
Best score : 0.9037900540991681
---------
RMSE train: 0.05638277444860054
RMSE test : 0.09656160461178102


In [147]:
pickle.dump(model_gboost, open('model_gboost.pickle','wb'))

### XGBOOST

In [185]:
model_xgb2 = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [186]:
model_xgb2.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4603, gamma=0.0468, learning_rate=0.05,
       max_delta_step=0, max_depth=3, min_child_weight=1.7817,
       missing=None, n_estimators=2200, n_jobs=1, nthread=-1,
       objective='reg:linear', random_state=7, reg_alpha=0.464,
       reg_lambda=0.8571, scale_pos_weight=1, seed=None, silent=1,
       subsample=0.5213)

In [187]:
print("RMSE train: {}".format(rmse(y_train, model_xgb2.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb2.predict(X_test))))

RMSE train: 0.08473900853570761
RMSE test : 0.10458834822873225


In [192]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.02,\
                                    max_depth=4,\
                                    min_child_weight=4,\
                                    n_estimators=2000,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=4, min_child_weight=4, missing=None, n_estimators=2000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=80,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [193]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.03567828886205884
RMSE test : 0.10063912394065101


In [194]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.02,\
                                    max_depth=4,\
                                    min_child_weight=4,\
                                    n_estimators=2100,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=4, min_child_weight=4, missing=None, n_estimators=2100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=80,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [195]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.03406941356859858
RMSE test : 0.10046979662764921


In [196]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.02,\
                                    max_depth=4,\
                                    min_child_weight=4,\
                                    n_estimators=2200,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=4, min_child_weight=4, missing=None, n_estimators=2200,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=80,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [197]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.03246630773962482
RMSE test : 0.1004689631772734


In [198]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.02,\
                                    max_depth=4,\
                                    min_child_weight=4,\
                                    n_estimators=2300,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=4, min_child_weight=4, missing=None, n_estimators=2300,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=80,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [199]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.03126735889741411
RMSE test : 0.10052931526435584


In [213]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.02,\
                                    max_depth=3,\
                                    min_child_weight=4,\
                                    n_estimators=2000,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=3, min_child_weight=4, missing=None, n_estimators=2000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=80,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [214]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.05450872848502246
RMSE test : 0.10103839632354322


2

In [218]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.02,\
                                    max_depth=4,\
                                    min_child_weight=4,\
                                    n_estimators=3000,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=4, min_child_weight=4, missing=None, n_estimators=3000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=80,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [219]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.02332591672056718
RMSE test : 0.1004016066531891


In [220]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.02,\
                                    max_depth=4,\
                                    min_child_weight=4,\
                                    n_estimators=3200,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=4, min_child_weight=4, missing=None, n_estimators=3200,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=80,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [221]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.02144233559968962
RMSE test : 0.10027159459816665


In [222]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.02,\
                                    max_depth=4,\
                                    min_child_weight=4,\
                                    n_estimators=3500,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=4, min_child_weight=4, missing=None, n_estimators=3500,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=80,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [223]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.01893085635256288
RMSE test : 0.100409591984285


In [231]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.02,\
                                    max_depth=4,\
                                    min_child_weight=4,\
                                    n_estimators=4000,\
                                    reg_alpha=0.04640,\
                                    reg_lambda=0.08571,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=4, min_child_weight=4, missing=None, n_estimators=4000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=80,
       reg_alpha=0.0464, reg_lambda=0.08571, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [232]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.010392488927132588
RMSE test : 0.10527876058694588


In [233]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.02,\
                                    max_depth=4,\
                                    min_child_weight=4,\
                                    n_estimators=4500,\
                                    reg_alpha=0.04640,\
                                    reg_lambda=0.08571,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=4, min_child_weight=4, missing=None, n_estimators=4500,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=80,
       reg_alpha=0.0464, reg_lambda=0.08571, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [234]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.009101887437862846
RMSE test : 0.10529143404088602


In [235]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.002,\
                                    max_depth=4,\
                                    min_child_weight=4,\
                                    n_estimators=4500,\
                                    reg_alpha=0.04640,\
                                    reg_lambda=0.08571,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.002,
       max_delta_step=0, max_depth=4, min_child_weight=4, missing=None,
       n_estimators=4500, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=80, reg_alpha=0.0464, reg_lambda=0.08571,
       scale_pos_weight=1, seed=None, silent=True, subsample=1)

In [236]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.07463032947395881
RMSE test : 0.10399087013005845


In [237]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.002,\
                                    max_depth=4,\
                                    min_child_weight=3,\
                                    n_estimators=4500,\
                                    reg_alpha=0.04640,\
                                    reg_lambda=0.08571,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.002,
       max_delta_step=0, max_depth=4, min_child_weight=3, missing=None,
       n_estimators=4500, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=80, reg_alpha=0.0464, reg_lambda=0.08571,
       scale_pos_weight=1, seed=None, silent=True, subsample=1)

In [238]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.07327920952710597
RMSE test : 0.10185357145223764


In [243]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.002,\
                                    max_depth=4,\
                                    min_child_weight=3,\
                                    n_estimators=5000,\
                                    reg_alpha=0.04640,\
                                    reg_lambda=0.08571,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.002,
       max_delta_step=0, max_depth=4, min_child_weight=3, missing=None,
       n_estimators=5000, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=80, reg_alpha=0.0464, reg_lambda=0.08571,
       scale_pos_weight=1, seed=None, silent=True, subsample=1)

In [244]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.07067937105125074
RMSE test : 0.10178697680959287


In [245]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.002,\
                                    max_depth=4,\
                                    min_child_weight=3,\
                                    n_estimators=5500,\
                                    reg_alpha=0.04640,\
                                    reg_lambda=0.08571,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.002,
       max_delta_step=0, max_depth=4, min_child_weight=3, missing=None,
       n_estimators=5500, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=80, reg_alpha=0.0464, reg_lambda=0.08571,
       scale_pos_weight=1, seed=None, silent=True, subsample=1)

In [246]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.06841636911054601
RMSE test : 0.10165499383350639


In [247]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.002,\
                                    max_depth=4,\
                                    min_child_weight=3,\
                                    n_estimators=6000,\
                                    reg_alpha=0.04640,\
                                    reg_lambda=0.08571,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.002,
       max_delta_step=0, max_depth=4, min_child_weight=3, missing=None,
       n_estimators=6000, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=80, reg_alpha=0.0464, reg_lambda=0.08571,
       scale_pos_weight=1, seed=None, silent=True, subsample=1)

In [248]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.06611684432492773
RMSE test : 0.10152059993040795


In [249]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.002,\
                                    max_depth=4,\
                                    min_child_weight=3,\
                                    n_estimators=6500,\
                                    reg_alpha=0.04640,\
                                    reg_lambda=0.08571,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.002,
       max_delta_step=0, max_depth=4, min_child_weight=3, missing=None,
       n_estimators=6500, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=80, reg_alpha=0.0464, reg_lambda=0.08571,
       scale_pos_weight=1, seed=None, silent=True, subsample=1)

In [250]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.0638569683045309
RMSE test : 0.10129835164345537


In [251]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.002,\
                                    max_depth=4,\
                                    min_child_weight=3,\
                                    n_estimators=7000,\
                                    reg_alpha=0.04640,\
                                    reg_lambda=0.08571,\
                                    nthread = -1,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.002,
       max_delta_step=0, max_depth=4, min_child_weight=3, missing=None,
       n_estimators=7000, n_jobs=1, nthread=-1, objective='reg:linear',
       random_state=80, reg_alpha=0.0464, reg_lambda=0.08571,
       scale_pos_weight=1, seed=None, silent=True, subsample=1)

In [252]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.06174250237964719
RMSE test : 0.10105149982210708


In [253]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.002,\
                                    max_depth=4,\
                                    min_child_weight=3,\
                                    n_estimators=7200,\
                                    reg_alpha=0.04640,\
                                    reg_lambda=0.08571,\
                                    nthread = -1,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.002,
       max_delta_step=0, max_depth=4, min_child_weight=3, missing=None,
       n_estimators=7200, n_jobs=1, nthread=-1, objective='reg:linear',
       random_state=80, reg_alpha=0.0464, reg_lambda=0.08571,
       scale_pos_weight=1, seed=None, silent=True, subsample=1)

In [254]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.06086721598085441
RMSE test : 0.10094192984630652


In [256]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.002,\
                                    max_depth=3,\ ##changed from 4 to 3
                                    min_child_weight=3,\
                                    n_estimators=7200,\
                                    reg_alpha=0.04640,\
                                    reg_lambda=0.08571,\
                                    nthread = -1,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.002,
       max_delta_step=0, max_depth=3, min_child_weight=3, missing=None,
       n_estimators=7200, n_jobs=1, nthread=-1, objective='reg:linear',
       random_state=80, reg_alpha=0.0464, reg_lambda=0.08571,
       scale_pos_weight=1, seed=None, silent=True, subsample=1)

In [257]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.07766532821485757
RMSE test : 0.10224574703311681


In [261]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.8,\
                                    gamma=0,\
                                    learning_rate=0.001,\
                                    max_depth=4,\
                                    min_child_weight=3,\
                                    n_estimators=7200,\
                                    reg_alpha=0.04640,\
                                    reg_lambda=0.08571,\
                                    nthread = -1,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.001,
       max_delta_step=0, max_depth=4, min_child_weight=3, missing=None,
       n_estimators=7200, n_jobs=1, nthread=-1, objective='reg:linear',
       random_state=80, reg_alpha=0.0464, reg_lambda=0.08571,
       scale_pos_weight=1, seed=None, silent=True, subsample=1)

In [262]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.07972196672416607
RMSE test : 0.10283660880838062


In [263]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.7,\
                                    gamma=0,\
                                    learning_rate=0.001,\
                                    max_depth=4,\
                                    min_child_weight=3,\
                                    n_estimators=7200,\
                                    reg_alpha=0.04640,\
                                    reg_lambda=0.08571,\
                                    nthread = -1,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learning_rate=0.001,
       max_delta_step=0, max_depth=4, min_child_weight=3, missing=None,
       n_estimators=7200, n_jobs=1, nthread=-1, objective='reg:linear',
       random_state=80, reg_alpha=0.0464, reg_lambda=0.08571,
       scale_pos_weight=1, seed=None, silent=True, subsample=1)

In [264]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.08009420923249856
RMSE test : 0.10210376447205215


In [265]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.6,\
                                    gamma=0,\
                                    learning_rate=0.001,\
                                    max_depth=4,\
                                    min_child_weight=3,\
                                    n_estimators=7200,\
                                    reg_alpha=0.04640,\
                                    reg_lambda=0.08571,\
                                    nthread = -1,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=0, learning_rate=0.001,
       max_delta_step=0, max_depth=4, min_child_weight=3, missing=None,
       n_estimators=7200, n_jobs=1, nthread=-1, objective='reg:linear',
       random_state=80, reg_alpha=0.0464, reg_lambda=0.08571,
       scale_pos_weight=1, seed=None, silent=True, subsample=1)

In [266]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.08045970987425934
RMSE test : 0.10176936313946804


In [267]:
model_xgb = xgb.XGBRegressor(\
                                    colsample_bytree=0.5,\
                                    gamma=0,\
                                    learning_rate=0.001,\
                                    max_depth=4,\
                                    min_child_weight=3,\
                                    n_estimators=7200,\
                                    reg_alpha=0.04640,\
                                    reg_lambda=0.08571,\
                                    nthread = -1,\
                                    random_state=rs_const)
model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0, learning_rate=0.001,
       max_delta_step=0, max_depth=4, min_child_weight=3, missing=None,
       n_estimators=7200, n_jobs=1, nthread=-1, objective='reg:linear',
       random_state=80, reg_alpha=0.0464, reg_lambda=0.08571,
       scale_pos_weight=1, seed=None, silent=True, subsample=1)

In [268]:
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

RMSE train: 0.0813878027335901
RMSE test : 0.10166554563307795


In [313]:
param_grid_xgb = {
                    'colsample_bytree' : [0.5,0.6],
                    'gamma': [0,1],
                    'subsample':[0.5,1],
                    'learning_rate':[0.001,0.05],
                    'max_depth': [2,3,4], 
                    'nrounds':[100],
                    'min_child_weight':[2,3,4],
                    'n_estimators':[7200],
                    'reg_alpha':[0.04640,0.5],
                    'reg_lamda':[0.08571,0.6],
}

In [314]:
gridsearch_xgb= GridSearchCV(estimator=xgb.XGBRegressor(random_state=rs_const), param_grid=param_grid_xgb,n_jobs=-1,\
                                                        cv=5)#,verbose=True)

In [ ]:
# use early_stopping_rounds to stop the cv when there is no score imporovement
#clf_XGB.fit(X_train,y_train, early_stopping_rounds=20, eval_set=[(X_test, y_test)], verbose=False)

In [315]:
gridsearch_xgb.fit(X_train, y_train,early_stopping_rounds=20, eval_set=[(X_test, y_test)], verbose=True)

[0]	validation_0-rmse:10.9532
Will train until validation_0-rmse hasn't improved in 20 rounds.
[1]	validation_0-rmse:10.4066
[2]	validation_0-rmse:9.88733
[3]	validation_0-rmse:9.39403
[4]	validation_0-rmse:8.92542
[5]	validation_0-rmse:8.48081
[6]	validation_0-rmse:8.0586
[7]	validation_0-rmse:7.65749
[8]	validation_0-rmse:7.27598
[9]	validation_0-rmse:6.91391
[10]	validation_0-rmse:6.56998
[11]	validation_0-rmse:6.24243
[12]	validation_0-rmse:5.93163
[13]	validation_0-rmse:5.63574
[14]	validation_0-rmse:5.35559
[15]	validation_0-rmse:5.09019
[16]	validation_0-rmse:4.83672
[17]	validation_0-rmse:4.59564
[18]	validation_0-rmse:4.36739
[19]	validation_0-rmse:4.14982
[20]	validation_0-rmse:3.94395
[21]	validation_0-rmse:3.74763
[22]	validation_0-rmse:3.56159
[23]	validation_0-rmse:3.3844
[24]	validation_0-rmse:3.21646
[25]	validation_0-rmse:3.05686
[26]	validation_0-rmse:2.9053
[27]	validation_0-rmse:2.76066
[28]	validation_0-rmse:2.62391
[29]	validation_0-rmse:2.49285
[30]	validation_0-

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=80,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'colsample_bytree': [0.5, 0.6], 'gamma': [0, 1], 'subsample': [0.5, 1], 'learning_rate': [0.001, 0.05], 'max_depth': [2, 3, 4], 'nrounds': [100], 'min_child_weight': [2, 3, 4], 'n_estimators': [7200], 'reg_alpha': [0.0464, 0.5], 'reg_lamda': [0.08571, 0.6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [316]:
print('Best params: {}'.format(gridsearch_xgb.best_params_))
print('Best score : {}'.format(gridsearch_xgb.best_score_))

model_xgb = gridsearch_xgb.best_estimator_
print("RMSE train: {}".format(rmse(y_train, model_xgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_xgb.predict(X_test))))

Best params: {'colsample_bytree': 0.6, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 4, 'min_child_weight': 4, 'n_estimators': 7200, 'nrounds': 100, 'reg_alpha': 0.0464, 'reg_lamda': 0.08571, 'subsample': 1}
Best score : 0.896841628013883
RMSE train: 0.07939488534471889
RMSE test : 0.10281161716758491


In [317]:
pickle.dump(model_xgb, open('model_xgb.pickle','wb'))

In [325]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV

In [327]:
# specify parameters and distributions to sample from
param_dist = {'reg_alpha':sp_randint(0.0464,1.0),
              'reg_lamda':sp_randint(0.0464,1.0),
             }

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(xgb.XGBRegressor(n_estimators=7200,colsample_bytree= 0.6), param_distributions=param_dist, cv=5,
                                   n_iter=n_iter_search, verbose=10, n_jobs=-1, random_state=80)


random_search.fit(X_train, y_train)
# report(random_search.cv_results_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 18.1min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 22.0min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 27.7min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 32.0min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 39.1min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 45.3min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 49.6min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=7200,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid='warn', n_iter=20, n_jobs=-1,
          param_distributions={'reg_alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10f91d780>, 'reg_lamda': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10f91da20>},
          pre_dispatch='2*n_jobs', random_state=80, refit=True,
          return_train_score='warn', scoring=None, verbose=10)

In [330]:
# report(random_search.cv_results_)

print ('Best Parameters: ', random_search.best_params_)

NameError: name 'report' is not defined

In [331]:
model_xgb = random_search.best_estimator_
print("RMSE train: {}".format(rmse(y_train, random_search.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  random_search.predict(X_test))))

RMSE train: 0.0024637632207474523
RMSE test : 0.10312515205172232


In [ ]:
gridsearch_xgb= GridSearchCV(estimator=xgb.XGBRegressor(random_state=rs_const), param_grid=param_grid_xgb,n_jobs=-1,\
                                                        cv=5)#,verbose=True)

In [334]:
# specify parameters and distributions to sample from
param_dist = {'n_estimators':sp_randint(7000,7500),
              'learning_rate':[0.05],
               'max_depth':[4],
               'min_child_weight':[4],
               'subsample':[1],
             }

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(xgb.XGBRegressor(), param_distributions=param_dist, cv=5,
                                   n_iter=n_iter_search, verbose=10, n_jobs=-1, random_state=80)


random_search.fit(X_train, y_train,early_stopping_rounds=20, eval_set=[(X_test, y_test)], verbose=True)


# report(random_search.cv_results_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   34.5s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   45.6s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  4.0min finished


[0]	validation_0-rmse:10.9532
Will train until validation_0-rmse hasn't improved in 20 rounds.
[1]	validation_0-rmse:10.4066
[2]	validation_0-rmse:9.88732
[3]	validation_0-rmse:9.39402
[4]	validation_0-rmse:8.92541
[5]	validation_0-rmse:8.48099
[6]	validation_0-rmse:8.05859
[7]	validation_0-rmse:7.65748
[8]	validation_0-rmse:7.27596
[9]	validation_0-rmse:6.91393
[10]	validation_0-rmse:6.56906
[11]	validation_0-rmse:6.24192
[12]	validation_0-rmse:5.93152
[13]	validation_0-rmse:5.63562
[14]	validation_0-rmse:5.35475
[15]	validation_0-rmse:5.08858
[16]	validation_0-rmse:4.83484
[17]	validation_0-rmse:4.5938
[18]	validation_0-rmse:4.36555
[19]	validation_0-rmse:4.14947
[20]	validation_0-rmse:3.94265
[21]	validation_0-rmse:3.74689
[22]	validation_0-rmse:3.561
[23]	validation_0-rmse:3.38382
[24]	validation_0-rmse:3.21562
[25]	validation_0-rmse:3.0562
[26]	validation_0-rmse:2.90418
[27]	validation_0-rmse:2.7595
[28]	validation_0-rmse:2.62296
[29]	validation_0-rmse:2.49195
[30]	validation_0-rm

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid='warn', n_iter=20, n_jobs=-1,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x11003b940>, 'learning_rate': [0.05], 'max_depth': [4], 'min_child_weight': [4], 'subsample': [1]},
          pre_dispatch='2*n_jobs', random_state=80, refit=True,
          return_train_score='warn', scoring=None, verbose=10)

In [335]:
model_xgb = random_search.best_estimator_
print("RMSE train: {}".format(rmse(y_train, random_search.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  random_search.predict(X_test))))

RMSE train: 0.0824840355256234
RMSE test : 0.10205096277974163


In [336]:
random_search.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=4, min_child_weight=4, missing=None, n_estimators=7175,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [345]:
# specify parameters and distributions to sample from
param_dist = {'n_estimators':sp_randint(7000,8000),
              'learning_rate':[0.001,0.05],
                'max_depth':[3,4],
               'min_child_weight':[3,4],
               'subsample':[1],
              'reg_alpha':sp_randint(0,1),
              'reg_lamda':sp_randint(0,1),     
             }


#'n_estimators':sp_randint(7000,7500)

n_iter_search = 20
random_search = RandomizedSearchCV(xgb.XGBRegressor(), param_distributions=param_dist, cv=5,
                                   n_iter=n_iter_search, verbose=10, n_jobs=-1, random_state=80)


random_search.fit(X_train, y_train,early_stopping_rounds=20, eval_set=[(X_test, y_test)], verbose=True)


# report(random_search.cv_results_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   44.9s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 19.9min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 24.2min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 37.7min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 46.5min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 55.0min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 64.6min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 73.7min finished


[0]	validation_0-rmse:10.9532
Will train until validation_0-rmse hasn't improved in 20 rounds.
[1]	validation_0-rmse:10.4066
[2]	validation_0-rmse:9.88732
[3]	validation_0-rmse:9.39402
[4]	validation_0-rmse:8.92541
[5]	validation_0-rmse:8.48099
[6]	validation_0-rmse:8.05859
[7]	validation_0-rmse:7.65748
[8]	validation_0-rmse:7.27596
[9]	validation_0-rmse:6.91393
[10]	validation_0-rmse:6.56906
[11]	validation_0-rmse:6.24192
[12]	validation_0-rmse:5.93152
[13]	validation_0-rmse:5.63562
[14]	validation_0-rmse:5.35475
[15]	validation_0-rmse:5.08858
[16]	validation_0-rmse:4.83484
[17]	validation_0-rmse:4.5938
[18]	validation_0-rmse:4.36555
[19]	validation_0-rmse:4.14947
[20]	validation_0-rmse:3.94265
[21]	validation_0-rmse:3.74689
[22]	validation_0-rmse:3.561
[23]	validation_0-rmse:3.38382
[24]	validation_0-rmse:3.21562
[25]	validation_0-rmse:3.0562
[26]	validation_0-rmse:2.90418
[27]	validation_0-rmse:2.7595
[28]	validation_0-rmse:2.62296
[29]	validation_0-rmse:2.49195
[30]	validation_0-rm

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid='warn', n_iter=20, n_jobs=-1,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x11539f6d8>, 'learning_rate': [0.001, 0.05], 'max_depth': [3, 4], 'min_child_weight': [3, 4], 'subsample': [1], 'reg_alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x11539fac8>, 'reg_lamda': <scipy.stats._distn_infrastructure.rv_frozen object at 0x115966080>},
          pre_dispatch='2*n_jobs', random_state=80, refit=True,
          return_train_score='warn

In [346]:
model_xgb = random_search.best_estimator_
print("RMSE train: {}".format(rmse(y_train, random_search.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  random_search.predict(X_test))))

RMSE train: 0.05322039103414972
RMSE test : 0.09770373035691089


In [347]:
random_search.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=4, min_child_weight=3, missing=None, n_estimators=7884,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, reg_lamda=0, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

#### Create Hyperparameter Search Space

#### Create Grid Search

In [168]:
gridsearch_lgb= GridSearchCV(estimator=lgb.LGBMRegressor(objective='regression',random_state=rs_const), param_grid=param_grid_lgb,n_jobs=-1, cv=5,verbose=True)

#### Fit The Model

In [169]:
gridsearch_lgb.fit(X_train, y_train)

Fitting 5 folds for each of 59049 candidates, totalling 295245 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   25.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   53.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed: 10.3min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed: 16.2min
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed: 19.8min
[Parallel(n_jobs=-1)]: Done 7192 tasks      | elapsed: 23.7min
[Parallel(n_jobs=-1)]: Done 8442 tasks      | elapsed: 27.8min
[Parallel(n_jobs=-1)]: Done 9792 tasks      | elapsed: 32.3min
[Parallel(n_jobs=-1)]: Done 11242 tasks      |

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, num_leaves=31, objective='regression',
       random_state=80, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'num_leaves': [4, 5, 6], 'learning_rate': [0.04, 0.05, 0.06], 'n_estimators': [500, 700, 900], 'max_bin': [40, 50, 60], 'bagging_fraction': [0.7, 0.8, 0.9], 'bagging_freq': [4, 5, 6], 'feature_fraction': [0.1, 0.2, 0.3], 'feature_fraction_seed': [7, 8, 9], 'bagging_seed': [7, 8, 9], 'min_data_in_leaf': [5, 6, 7], 'min_sum_hessian_in_leaf': [11]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None,

#### Evaluate The Model And Print The Results

In [170]:
print("_______________________________________")
print("lgb: Regression")
print("_______________________________________")
print('Best params: {}'.format(gridsearch_lgb.best_params_))
print('Best score : {}'.format(gridsearch_lgb.best_score_))
print("---------")
model_lgb = gridsearch_lgb.best_estimator_
print("RMSE train: {}".format(rmse(y_train, model_lgb.predict(X_train))))
print("RMSE test : {}".format(rmse(y_test,  model_lgb.predict(X_test))))

_______________________________________
lgb: Regression
_______________________________________
Best params: {'bagging_fraction': 0.9, 'bagging_freq': 4, 'bagging_seed': 9, 'feature_fraction': 0.1, 'feature_fraction_seed': 7, 'learning_rate': 0.04, 'max_bin': 40, 'min_data_in_leaf': 6, 'min_sum_hessian_in_leaf': 11, 'n_estimators': 700, 'num_leaves': 6}
Best score : 0.896827843999088
---------
RMSE train: 0.08493187271043416
RMSE test : 0.10346124046188622


In [171]:
pickle.dump(model_lgb, open('model_lgb.pickle','wb'))

In [ ]:
############################################################

### 6. MLP Regression

#### Create Hyperparameter Search Space

In [152]:
param_grid_mlp = {'alpha': [2,2.25,2.5,2.75,3,3.25,3.5,3.75,4,4.25,4.50,5.0],
              'activation': ['tanh'],
               'hidden_layer_sizes' : [(10,10),(10,5), (10,20),(20,10),(20,20),(20,30),(30,30),(30,20),(20,30),(40,40),(40,30),(50,50),(60,60),(70,70),(80,80),(90,90)],
               'solver' : ['lbfgs'],
                'learning_rate':['adaptive'],
                'learning_rate_init':[0.003]
               }


#### Create Grid Search

In [153]:
gridsearch_mlp = GridSearchCV(MLPRegressor(random_state=rs_const), param_grid_mlp, n_jobs=-1,cv=5, verbose=True)

#### Fit The Model

In [154]:
gridsearch_mlp.fit(X_train_std,y_train)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   50.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed: 14.7min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=80, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': [2, 2.25, 2.5, 2.75, 3, 3.25, 3.5, 3.75, 4, 4.25, 4.5, 5.0], 'activation': ['tanh'], 'hidden_layer_sizes': [(10, 10), (10, 5), (10, 20), (20, 10), (20, 20), (20, 30), (30, 30), (30, 20), (20, 30), (40, 40), (40, 30), (50, 50), (60, 60), (70, 70), (80, 80), (90, 90)], 'solver': ['lbfgs'], 'learning_rate': ['adaptive'], 'learning_rate_init': [0.003]},
       pre_dispatch='2*n_jobs', refit=True, return_

#### Evaluate The Model And Print The Results

In [158]:
print("_______________________________________")
print("MLP: Regression")
print("_______________________________________")
print('Best params: {}'.format(gridsearch_mlp.best_params_))
print('Best score : {}'.format(gridsearch_mlp.best_score_))
print("---------")
model_mlp = gridsearch_mlp.best_estimator_
print("RMSE train: {}".format(rmse(y_train, model_mlp.predict(X_train_std))))
print("RMSE test : {}".format(rmse(y_test,  model_mlp.predict(X_test_std))))

_______________________________________
MLP: Regression
_______________________________________
Best params: {'activation': 'tanh', 'alpha': 5.0, 'hidden_layer_sizes': (20, 10), 'learning_rate': 'adaptive', 'learning_rate_init': 0.003, 'solver': 'lbfgs'}
Best score : 0.8817854092919377
---------
RMSE train: 0.07873849482402347
RMSE test : 0.1067523028132266


In [157]:
import pickle

pickle.dump(model_mlp, open('model_mlp.pickle','wb'))

In [ ]:
####################

### KERAS

In [216]:
from keras.layers import Dense
from keras.models import Sequential
from keras.regularizers import l1
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/imp.py", line 342, in load_dynamic
    return _load(s

In [ ]:
# X_train = StandardScaler().fit_transform(X_train)

In [ ]:
# X_tr, X_val, y_tr, y_val = train_test_split(X_train, y, random_state = 3)

In [ ]:
# X_tr.shape

In [ ]:
# X_tr

In [ ]:
#model = Sequential()
##model.add(Dense(256, activation="relu", input_dim = X_train.shape[1]))
#model.add(Dense(1, input_dim = X_train.shape[1], W_regularizer=l1(0.001)))

#model.compile(loss = "mse", optimizer = "adam")

In [ ]:
# model.summary()

In [ ]:
# hist = model.fit(X_tr, y_tr, validation_data = (X_val, y_val))

In [ ]:
# pd.Series(model.predict(X_val)[:,0]).hist()